In [ ]:
!pip install -U setuptools wheel
!pip install autogluon

     |████████████████████████████████| 1.2 MB 5.3 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 59 kB 3.1 MB/s 
     |████████████████████████████████| 267 kB 10.2 MB/s 
     |████████████████████████████████| 48 kB 4.8 MB/s 
     |████████████████████████████████| 189 kB 46.7 MB/s 
     |████████████████████████████████| 162 kB 49.6 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.0 MB 45.1 MB/s 
     |████████████████████████████████| 802 kB 47.4 MB/s 
     |████████████████████████████████| 132 kB 38.6 MB/s 
     |████████████████████████████████| 59.6 MB 1.3 MB/s 
     |████████████████████████████████| 296 kB 52.2 MB/s 
     |████████████████████████████████| 166.7 MB 18 kB/s 
     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:37tcmalloc: large alloc 1147494400 bytes == 0x3244000 @  0x7f87808c0615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98

NameError: ignored

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# adapted from https://towardsdatascience.com/pytorch-lstms-for-time-series-data-cd16190929d7

class LSTM(nn.Module):
    def __init__(self, hidden_layers=64):
        input_size = 7
        super(LSTM, self).__init__()
        self.hidden_layers = hidden_layers
        # lstm1, lstm2, linear are all layers in the network
        self.lstm1 = nn.LSTMCell(input_size, self.hidden_layers)
        self.lstm2 = nn.LSTMCell(self.hidden_layers, self.hidden_layers)
        self.linear = nn.Linear(self.hidden_layers, 1)
        
    def forward(self, y, test_inputs=()):
        outputs, n_samples, n_features = [], y.size(0), y.size(2)
        h_t = torch.zeros(n_samples, self.hidden_layers, dtype=torch.float32)
        c_t = torch.zeros(n_samples, self.hidden_layers, dtype=torch.float32)
        h_t2 = torch.zeros(n_samples, self.hidden_layers, dtype=torch.float32)
        c_t2 = torch.zeros(n_samples, self.hidden_layers, dtype=torch.float32)
        
        for time_step in y.split(1, dim=1):
            # N, 1
            time_step = torch.squeeze(time_step, dim=1)
            h_t, c_t = self.lstm1(time_step.float(), (h_t.float(), c_t.float())) # initial hidden and cell states
            #h_t2, c_t2 = self.lstm2(h_t.float(), (h_t2.float(), c_t2.float())) # new hidden and cell states
            output = self.linear(h_t.float()) # output from the last FC layer
            outputs.append(output)
            
        for tester in test_inputs:
            # this only generates future predictions if we pass in future_preds>0
            # mirrors the code above, using last output/prediction as input
            h_t, c_t = self.lstm1(tester.float(), (h_t.float(), c_t.float()))
            #h_t2, c_t2 = self.lstm2(h_t.float(), (h_t2.float(), c_t2.float()))
            output = self.linear(h_t.float())
            outputs.append(output)
        # transform list to tensor    
        outputs = torch.cat(outputs, dim=1)
        return outputs


def training_loop(n_epochs, model, optimiser, loss_fn, 
                  train_input, train_target, test_input, test_target):
    for i in range(n_epochs):
        def closure():
            optimiser.zero_grad()
            out = model(train_input.float())
            loss = loss_fn(out, torch.squeeze(train_target, 2))
            loss.backward()
            return loss
        optimiser.step(closure)
        with torch.no_grad():
            pred = model(test_input)
            # use all pred samples, but only go to 999
            loss = loss_fn(pred, torch.squeeze(test_target, 2))
            print("TEST LOSS: " + str(loss))
            y = pred.detach().numpy()
        # draw figures
        '''plt.figure(figsize=(12,6))
        plt.title(f"Step {i+1}")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        n = train_input.shape[1] # 999
        def draw(yi, colour):
            plt.plot(np.arange(n), yi[:n], colour, linewidth=2.0)
            plt.plot(np.arange(n, n+future), yi[n:], colour+":", linewidth=2.0)
        draw(y[0], 'r')
        draw(y[1], 'b')
        draw(y[2], 'g')
        plt.savefig("predict%d.png"%i, dpi=200)
        plt.close()'''
        # print the loss
        out = model(train_input)
        loss_print = loss_fn(out, torch.squeeze(train_target, 2))
        print("Step: {}, Loss: {}".format(i, loss_print))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/CryptoDataFull.zip

Archive:  /content/gdrive/MyDrive/CryptoDataFull.zip
   creating: CryptoDataFull/
  inflating: CryptoDataFull/allcrypto_postsToEnd2021.csv  
  inflating: CryptoDataFull/allcrypto_postsToEnd2020.csv  
  inflating: CryptoDataFull/dogeMarSep21.csv  
  inflating: __MACOSX/CryptoDataFull/._dogeMarSep21.csv  
  inflating: CryptoDataFull/Transformation.py  
  inflating: __MACOSX/CryptoDataFull/._Transformation.py  
  inflating: CryptoDataFull/ethTrends.csv  
  inflating: __MACOSX/CryptoDataFull/._ethTrends.csv  
  inflating: CryptoDataFull/etherAugFeb21.csv  
  inflating: __MACOSX/CryptoDataFull/._etherAugFeb21.csv  
  inflating: CryptoDataFull/bitcoinTrends.csv  
  inflating: __MACOSX/CryptoDataFull/._bitcoinTrends.csv  
  inflating: CryptoDataFull/GLMBitcoin.ipynb  
  inflating: CryptoDataFull/VisualizeR^2Dec.ipynb  
  inflating: CryptoDataFull/EtherWorldAugFeb21.csv  
  inflating: __MACOSX/CryptoDataFull/._EtherWorldAugFeb21.csv  
  inflating: CryptoDataFull/allOctFeb22.csv  
  inflating: 

In [ ]:
import os
os.chdir("CryptoDataFull")

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

pd.options.mode.chained_assignment = None
allList = ["standardized_txs", "standardized_posts", "standardized_tweets", "standardized_trends", 
   "standardized_allTweets", "standardized_allPosts",
    "standardized_allTrends", "standardized_price"]
import DataCleaning
bitcoinDataFull = DataCleaning.getBitcoinData()
bitcoinDataTrainInput = bitcoinDataFull[0:456][allList] #excluding final month
bitcoinDataTrainOutput = bitcoinDataFull[0:456][["standardized_price"]]
bitcoinDataTestInput = bitcoinDataFull[456:][allList] #final month
bitcoinDataTestOutput = bitcoinDataFull[456:][["standardized_price"]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DtypeWarning: Columns (83,87,88,91) have mixed types.Specify dtype option on import or set low_memory=False.
  
/content/CryptoDataFull/DataCleaning.py:225: DtypeWarning: Columns (83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([bitcoinDailyTwts, bitcoinDailyPrice, bitcoinDailyPosts, bitcoinModTrends, bitcoinDailyTxs, getAllData()], axis=1)
/content/CryptoDataFull/DataCleaning.py:225: DtypeWarning: Columns (82,84,86,87,88) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([bitcoinDailyTwts, bitcoinDailyPrice, bitcoinDailyPosts, bitcoinModTrends, bitcoinDailyTxs, getAllData()], axis=1)
/content/CryptoDataFull/DataCleaning.py:225: DtypeWarning: Columns (11,22,78,85,88,89,90,91,92,93,94) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([bitcoinDailyTwts, bitcoinDailyPrice, bitcoinDai

Index(['allPosts'], dtype='object')


In [ ]:
train_data = TabularDataset(bitcoinDataTrainInput)
predictor_age2 = TabularPredictor(label="standardized_price", path = "newmodel", eval_metric="r2").fit(train_data, time_limit=60)


Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "newmodel/"
AutoGluon Version:  0.4.1
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    456
Train Data Columns: 7
Label Column: standardized_price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1.7003375300983339, -1.705674807055476, -0.15899, 1.03505)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11886.65 MB
	Train Data (Original)  Memory Usage: 0.03 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set featu

In [ ]:
TabularPredictor.load("newmodel/").evaluate(bitcoinDataTestInput)

Evaluation: r2 on test data: -0.9045412720337327
Evaluations on test data:
{
    "r2": -0.9045412720337327,
    "root_mean_squared_error": -0.7261277353154488,
    "mean_squared_error": -0.5272614879943428,
    "mean_absolute_error": -0.6376364297682067,
    "pearsonr": 0.5060199670192783,
    "median_absolute_error": -0.6754447929891569
}


{'mean_absolute_error': -0.6376364297682067,
 'mean_squared_error': -0.5272614879943428,
 'median_absolute_error': -0.6754447929891569,
 'pearsonr': 0.5060199670192783,
 'r2': -0.9045412720337327,
 'root_mean_squared_error': -0.7261277353154488}

In [ ]:
allList = ["standardized_txs", "standardized_posts", "standardized_tweets", "standardized_trends", 
   "standardized_allTweets", "standardized_allPosts",
    "standardized_allTrends"]
bitcoinDataTrainInput = bitcoinDataFull[0:548][allList] #excluding final month
bitcoinDataTrainOutput = bitcoinDataFull[0:548][["standardized_price"]]
bitcoinDataTestInput = bitcoinDataFull[548:][allList] #final month
bitcoinDataTestOutput = bitcoinDataFull[548:][["standardized_price"]]

In [ ]:
bitcoinDataTrainInput

,standardized_txs,standardized_posts,standardized_tweets,standardized_trends,standardized_allTweets,standardized_allPosts,standardized_allTrends
2020-08-02,0.234097,-1.283629,-1.237362,-0.736154,-1.075424,-1.276847,-1.094051
2020-08-03,1.244680,-1.401168,-1.076478,-0.975663,-0.867565,-1.275718,-1.117793
2020-08-04,1.140702,-1.332028,-1.115481,-1.095418,-0.907538,-1.266688,-1.165276
2020-08-05,0.320985,-1.214488,-1.032601,-1.035540,-0.923527,-1.282491,-1.165276
2020-08-06,1.748144,-1.159176,-1.013100,-0.975663,-0.955505,-1.254271,-1.141535
...,...,...,...,...,...,...,...
2022-01-27,-0.469461,-0.274174,0.600615,0.306034,1.458858,-0.003594,0.389799
2022-01-28,-0.278198,-0.523081,0.673744,0.187088,1.314956,-0.092767,0.348252
2022-01-29,-1.017590,-0.391714,-0.111175,-0.090451,0.315633,-0.172910,0.348252
2022-01-30,-1.543211,-0.626792,-0.189179,-0.249045,0.051812,-0.207901,0.265156


In [ ]:
torch.manual_seed(123)
np.random.seed(123)
new_model = LSTM()
criterion = nn.MSELoss()
optimiser = torch.optim.Adam(new_model.parameters(), lr=0.01)
training_loop(n_epochs = 300,
              model = new_model,
              optimiser = optimiser,
              loss_fn = criterion,
              train_input = torch.tensor(bitcoinDataTrainInput.values.astype(np.float32)).unsqueeze(0),
              train_target = torch.tensor(bitcoinDataTrainOutput.values.astype(np.float32)).unsqueeze(0),
              test_input = torch.tensor(bitcoinDataTestInput.values.astype(np.float32)).unsqueeze(0),
              test_target = torch.tensor(bitcoinDataTestOutput.values.astype(np.float32)).unsqueeze(0))

NameError: ignored

In [ ]:
shibaDataFull = DataCleaning.getShibaData()
shibaDataTrainInput = shibaDataFull[0:456][allList] #excluding final month
shibaDataTrainOutput = shibaDataFull[0:456][["standardized_price"]]
shibaDataTestInput = shibaDataFull[456:][allList] #final month
shibaDataTestOutput = shibaDataFull[456:][["standardized_price"]]

/content/CryptoDataFull/DataCleaning.py:102: DtypeWarning: Columns (83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([dailyTwts, dailyPrice, dailyPosts, dailyTxs, modTrends, getAllData()], axis=1)
/content/CryptoDataFull/DataCleaning.py:102: DtypeWarning: Columns (82,84,86,87,88) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([dailyTwts, dailyPrice, dailyPosts, dailyTxs, modTrends, getAllData()], axis=1)
/content/CryptoDataFull/DataCleaning.py:102: DtypeWarning: Columns (11,22,78,85,88,89,90,91,92,93,94) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([dailyTwts, dailyPrice, dailyPosts, dailyTxs, modTrends, getAllData()], axis=1)
/content/CryptoDataFull/DataCleaning.py:102: DtypeWarning: Columns (81,82,83,87,88,89) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([dailyTwts, dailyPrice, dailyPos

Index(['allPosts'], dtype='object')


In [ ]:
train_data = TabularDataset(shibaDataTrainInput)
predictor_age3 = TabularPredictor(label="standardized_price", path = "newmodel2", eval_metric="r2").fit(train_data, time_limit=60)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "newmodel2/"
AutoGluon Version:  0.4.1
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    456
Train Data Columns: 7
Label Column: standardized_price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (4.328056225426675, -0.710263066905105, -0.3915, 0.6364)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11550.91 MB
	Train Data (Original)  Memory Usage: 0.03 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set fe

In [ ]:
TabularPredictor.load("newmodel2/").evaluate(shibaDataTestInput)

Evaluation: r2 on test data: -0.5873478822793081
Evaluations on test data:
{
    "r2": -0.5873478822793081,
    "root_mean_squared_error": -0.8625035141561688,
    "mean_squared_error": -0.7439123119317397,
    "mean_absolute_error": -0.8165629131398799,
    "pearsonr": 0.9080039381151497,
    "median_absolute_error": -0.795024488144618
}


{'mean_absolute_error': -0.8165629131398799,
 'mean_squared_error': -0.7439123119317397,
 'median_absolute_error': -0.795024488144618,
 'pearsonr': 0.9080039381151497,
 'r2': -0.5873478822793081,
 'root_mean_squared_error': -0.8625035141561688}

In [ ]:
shiba_model = LSTM()
criterion = nn.MSELoss()
optimiser = torch.optim.Adam(shiba_model.parameters(), lr=0.01)
training_loop(n_epochs = 100,
              model = shiba_model,
              optimiser = optimiser,
              loss_fn = criterion,
              train_input = torch.tensor(shibaDataTrainInput.values.astype(np.float32)).unsqueeze(0),
              train_target = torch.tensor(shibaDataTrainOutput.values.astype(np.float32)).unsqueeze(0),
              test_input = torch.tensor(shibaDataTestInput.values.astype(np.float32)).unsqueeze(0),
              test_target = torch.tensor(shibaDataTestOutput.values.astype(np.float32)).unsqueeze(0))

TEST LOSS: tensor(0.8882)
Step: 0, Loss: 0.7903334498405457
TEST LOSS: tensor(0.7657)
Step: 1, Loss: 0.540715754032135
TEST LOSS: tensor(0.4824)
Step: 2, Loss: 0.3224453628063202
TEST LOSS: tensor(0.0346)
Step: 3, Loss: 0.3807208240032196
TEST LOSS: tensor(0.0888)
Step: 4, Loss: 0.2202317714691162
TEST LOSS: tensor(0.3347)
Step: 5, Loss: 0.1580784022808075
TEST LOSS: tensor(0.4644)
Step: 6, Loss: 0.14665119349956512
TEST LOSS: tensor(0.4171)
Step: 7, Loss: 0.12487724423408508
TEST LOSS: tensor(0.2214)
Step: 8, Loss: 0.08039413392543793
TEST LOSS: tensor(0.0438)
Step: 9, Loss: 0.08412238210439682
TEST LOSS: tensor(0.0647)
Step: 10, Loss: 0.0939771756529808
TEST LOSS: tensor(0.0403)
Step: 11, Loss: 0.07047351449728012
TEST LOSS: tensor(0.0574)
Step: 12, Loss: 0.06976422667503357
TEST LOSS: tensor(0.0780)
Step: 13, Loss: 0.0759611651301384
TEST LOSS: tensor(0.0751)
Step: 14, Loss: 0.07395149767398834
TEST LOSS: tensor(0.0561)
Step: 15, Loss: 0.06609021127223969
TEST LOSS: tensor(0.0390)
S

In [ ]:
dogeDataFull = DataCleaning.getDogeData()
dogeDataTrainInput = dogeDataFull[0:456][allList] #excluding final month
dogeDataTrainOutput = dogeDataFull[0:456][["standardized_price"]]
dogeDataTestInput = dogeDataFull[456:][allList] #final month
dogeDataTestOutput = dogeDataFull[456:][["standardized_price"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (88) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/content/CryptoDataFull/DataCleaning.py:165: DtypeWarning: Columns (83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([dailyTwts, dailyPrice, dailyPosts, dailyTxs, modTrends, getAllData()], axis=1)
/content/CryptoDataFull/DataCleaning.py:165: DtypeWarning: Columns (82,84,86,87,88) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([dailyTwts, dailyPrice, dailyPosts, dailyTxs, modTrends, getAllData()], axis=1)
/content/CryptoDataFull/DataCleaning.py:165: DtypeWarning: Colu

Index(['allPosts'], dtype='object')


In [ ]:
train_data = TabularDataset(dogeDataTrainInput)
predictor_age3 = TabularPredictor(label="standardized_price", path = "newmodel3", eval_metric="r2").fit(train_data)
TabularPredictor.load("newmodel3/").evaluate(dogeDataTestInput)

Beginning AutoGluon training ...
AutoGluon will save models to "newmodel3/"
AutoGluon Version:  0.4.1
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    456
Train Data Columns: 7
Label Column: standardized_price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (4.047161173017568, -1.076357364208957, -0.06782, 1.1035)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11471.68 MB
	Train Data (Original)  Memory Usage: 0.03 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

{'mean_absolute_error': -0.2844918584008436,
 'mean_squared_error': -0.11925744554861871,
 'median_absolute_error': -0.25670112083861707,
 'pearsonr': 0.740415595407449,
 'r2': -0.20497076747394116,
 'root_mean_squared_error': -0.3453367132938789}

In [ ]:
doge_model = LSTM()
criterion = nn.MSELoss()
optimiser = torch.optim.Adam(doge_model.parameters(), lr=0.01)
training_loop(n_epochs = 30,
              model = doge_model,
              optimiser = optimiser,
              loss_fn = criterion,
              train_input = torch.tensor(dogeDataTrainInput.values.astype(np.float32)).unsqueeze(0),
              train_target = torch.tensor(dogeDataTrainOutput.values.astype(np.float32)).unsqueeze(0),
              test_input = torch.tensor(dogeDataTestInput.values.astype(np.float32)).unsqueeze(0),
              test_target = torch.tensor(dogeDataTestOutput.values.astype(np.float32)).unsqueeze(0))

TEST LOSS: tensor(0.0178)
Step: 0, Loss: 0.771108090877533
TEST LOSS: tensor(0.0137)
Step: 1, Loss: 0.5029100775718689
TEST LOSS: tensor(0.0163)
Step: 2, Loss: 0.29050835967063904
TEST LOSS: tensor(0.0596)
Step: 3, Loss: 0.36671000719070435
TEST LOSS: tensor(0.0594)
Step: 4, Loss: 0.3310298025608063
TEST LOSS: tensor(0.0259)
Step: 5, Loss: 0.23060977458953857
TEST LOSS: tensor(0.0127)
Step: 6, Loss: 0.20813709497451782
TEST LOSS: tensor(0.0084)
Step: 7, Loss: 0.21350792050361633
TEST LOSS: tensor(0.0069)
Step: 8, Loss: 0.20858928561210632
TEST LOSS: tensor(0.0065)
Step: 9, Loss: 0.18373385071754456
TEST LOSS: tensor(0.0071)
Step: 10, Loss: 0.14651402831077576
TEST LOSS: tensor(0.0084)
Step: 11, Loss: 0.12446318566799164
TEST LOSS: tensor(0.0067)
Step: 12, Loss: 0.12856870889663696
TEST LOSS: tensor(0.0055)
Step: 13, Loss: 0.10831144452095032
TEST LOSS: tensor(0.0161)
Step: 14, Loss: 0.08985543251037598
TEST LOSS: tensor(0.0365)
Step: 15, Loss: 0.08969386667013168
TEST LOSS: tensor(0.05

In [ ]:
ethDataFull = DataCleaning.getEthereumData()
ethDataTrainInput = ethDataFull[0:456][allList] #excluding final month
ethDataTrainOutput = ethDataFull[0:456][["standardized_price"]]
ethDataTestInput = ethDataFull[456:][allList] #final month
ethDataTestOutput = ethDataFull[456:][["standardized_price"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (83,86,87,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/content/CryptoDataFull/DataCleaning.py:286: DtypeWarning: Columns (83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([dailyTwts, dailyPrice, dailyPosts, dailyTxs, modTrends, getAllData()], axis=1)
/content/CryptoDataFull/DataCleaning.py:286: DtypeWarning: Columns (82,84,86,87,88) have mixed types.Specify dtype option on import or set low_memory=False.
  allData = pd.concat([dailyTwts, dailyPrice, dailyPosts, dailyTxs, modTrends, getAllData()], axis=1)
/content/CryptoDataFull/DataCleaning.py:28

Index(['allPosts'], dtype='object')


In [ ]:
train_data = TabularDataset(ethDataTrainInput)
predictor_age4 = TabularPredictor(label="standardized_price", path = "newmodel4", eval_metric="r2").fit(train_data)
TabularPredictor.load("newmodel4/").evaluate(ethDataTestInput)

Beginning AutoGluon training ...
AutoGluon will save models to "newmodel4/"
AutoGluon Version:  0.4.1
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    456
Train Data Columns: 7
Label Column: standardized_price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1.6832649724107107, -1.4051938863963773, -0.28354, 0.89559)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11445.21 MB
	Train Data (Original)  Memory Usage: 0.03 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in 

{'mean_absolute_error': -0.3587203231050805,
 'mean_squared_error': -0.18911491211322332,
 'median_absolute_error': -0.34140002166049394,
 'pearsonr': 0.5954065462662949,
 'r2': 0.35313278840963835,
 'root_mean_squared_error': -0.4348734437893667}

In [ ]:
eth_model = LSTM()
criterion = nn.MSELoss()
optimiser = torch.optim.Adam(eth_model.parameters(), lr=0.005)
training_loop(n_epochs = 75,
              model = eth_model,
              optimiser = optimiser,
              loss_fn = criterion,
              train_input = torch.tensor(ethDataTrainInput.values.astype(np.float32)).unsqueeze(0),
              train_target = torch.tensor(ethDataTrainOutput.values.astype(np.float32)).unsqueeze(0),
              test_input = torch.tensor(ethDataTestInput.values.astype(np.float32)).unsqueeze(0),
              test_target = torch.tensor(ethDataTestOutput.values.astype(np.float32)).unsqueeze(0))

TEST LOSS: tensor(0.2962)
Step: 0, Loss: 0.8886749148368835
TEST LOSS: tensor(0.2688)
Step: 1, Loss: 0.7133722901344299
TEST LOSS: tensor(0.2357)
Step: 2, Loss: 0.5363751649856567
TEST LOSS: tensor(0.1922)
Step: 3, Loss: 0.36924293637275696
TEST LOSS: tensor(0.1326)
Step: 4, Loss: 0.2939853072166443
TEST LOSS: tensor(0.0621)
Step: 5, Loss: 0.27756738662719727
TEST LOSS: tensor(0.0206)
Step: 6, Loss: 0.2140713483095169
TEST LOSS: tensor(0.0583)
Step: 7, Loss: 0.17181719839572906
TEST LOSS: tensor(0.2109)
Step: 8, Loss: 0.19390086829662323
TEST LOSS: tensor(0.2944)
Step: 9, Loss: 0.20551317930221558
TEST LOSS: tensor(0.2641)
Step: 10, Loss: 0.1853628009557724
TEST LOSS: tensor(0.1959)
Step: 11, Loss: 0.16148586571216583
TEST LOSS: tensor(0.1387)
Step: 12, Loss: 0.148185595870018
TEST LOSS: tensor(0.1030)
Step: 13, Loss: 0.1433713287115097
TEST LOSS: tensor(0.0844)
Step: 14, Loss: 0.14165018498897552
TEST LOSS: tensor(0.0774)
Step: 15, Loss: 0.14020952582359314
TEST LOSS: tensor(0.0790)
S